In [ ]:
import import_ipynb
import sys
from openai import OpenAI
#import demoA.test1 as test1
#from DoctorAppointment import *
import datetime

from ipynb.fs.full.DoctorAppointmentGit import get_doctor_by_specialist, get_doctor_availability,get_all_doctor,tool


import json

GPT_MODEL = 'gpt-4.1'

openai_api_key = ""
client = OpenAI(api_key=openai_api_key)

current_datetime = datetime.datetime.now()

initial_prompt = {'role':"system",
                  "content":f"""
                                1. You are a helpful assistant designed to book appointments with doctors
                                2. You need to take all info that is relevant to booking
                                3. Ask about their symptoms and preferred doctors and availabity 
                                4. Give them appointment details in a reciept form 
                                5. Answer in in precise manner dont give long answers or questions
                                6. do not send any irrelevant info cause health care is a serious topic dont generate names of deaseas and doctors by your own
                                7. User's answer not releavent ask them to try with different words
                                8. Consider only India IST date after "{current_datetime}". Do not book on Sunday and Indian holidays

"""}

##handle history
messages = [initial_prompt]

#color
green = '\033[92m'
red = '\033[91m'
reset = '\033[0m'


global isStart

isStart = True



# define role 
initial_prompt = {'role':"system",
                  "content":f"""
                                1. You are a helpful assistant designed to book appointments with doctors
                                2. You need to take all info that is relevant to booking
                                3. Ask about their symptoms and preferred doctors and availabity 
                                4. Give them appointment details in a reciept form 
                                5. Answer in in precise manner dont give long answers or questions
                                6. do not send any irrelevant info cause health care is a serious topic dont generate names of deaseas and doctors by your own
                                7. User's answer not releavent ask them to try with different words
                                8. Consider only India IST date after "{current_datetime}". Do not book on Sunday and Indian holidays

"""}

def handle_response(user_input):
    global isStart;
    isStart = True
    global messages
    messages.append({'role':'user', "content":user_input})
    response = client.chat.completions.create(
        model = GPT_MODEL,
        messages = messages,
        tools = tool,
        tool_choice="auto")
    response_message = response.choices[0].message
    
    messages.append(response_message)
    indexBye = -1
    indexAp = -1
    indexCm = -1
    indexTc = -1
    if response_message.content != None:
       indexBye = response_message.content.find("Good bye!")
       indexCm = response_message.content.find("confirmed") 
       indexTc = response_message.content.find("Take care")
       indexAp = response_message.content.find("Appointment booking complete")
    if indexBye != -1 or indexAp != -1 or indexCm != -1 or indexTc != -1:
        isStart = False
   
    print(f'{green}AI: {response_message.content}{reset}')
   

# tool functionality 
    tool_calls = response_message.tool_calls
    
    if tool_calls:
        for tool_call in tool_calls:
            tool_call_id = tool_call.id 
            tool_function_name = tool_call.function.name 
            tool_query  = json.loads(tool_call.function.arguments)
            Q = True
            
            if tool_function_name == 'get_all_doctor':
                results  = get_doctor(tool_query['query'])
            elif tool_function_name == 'get_doctor_by_specialist':
                results = get_doctor_by_specialist(tool_query['query'])
            elif tool_function_name == 'get_doctor_availability':
                results = get_doctor_availability(tool_query['doctor_name'])
            else:
                results = get_unable_to_find(tool_query['query'])
                #results = 'error'
                Q = False
                isStart = False
            
            
            if Q:
                messages.append({
                    "role": "tool", 
                    "tool_call_id": tool_call_id, 
                    "name": tool_function_name, 
                    "content": results
                })
                
            else: 
                print(f"Error: function {tool_function_name} does not exist")
                isStart = False

        model_response_with_function_call = client.chat.completions.create(
            model=GPT_MODEL,
            messages=messages,
        )

        messages.append(model_response_with_function_call.choices[0].message)
        print(f"{green}AI: {model_response_with_function_call.choices[0].message.content}{reset}")






#intial question

print(f"{green}AI: Hello! I can help you book an appointment with a doctor. Can you please tell me what symptoms you're experiencing?{reset}")

# the flow of converation

while isStart != False:
    user_input = input('YOU: ')
  
    if isStart != False:
      handle_response(user_input)
#    if isStart == False:
      exit()


AI: Hello! I can help you book an appointment with a doctor. Can you please tell me what symptoms you're experiencing?


YOU:  teeth pain


AI: None
AI: For teeth pain, would you like to book an appointment with Dr. Tom Crusie (Dentist)? Please share your preferred date and time (except Sundays and holidays), and your availability.


YOU:  10th June 2025 at 10AM


AI: The selected date, 10th June 2025, is before today's allowed date. Please provide a new date after 22nd June 2025 for booking.


YOU:  sure 23rd June 2025 10 AM


AI: None
Tuesday to Saturday, 10 AM - 6 PM
AI: Dr. Tom Crusie (Dentist) is available on 23rd June 2025 at 10 AM. Please confirm your full name and contact number to complete the booking.


YOU:  Venkatesh Krishnakumar


In [ ]:
pip install --upgrade openai